In [1]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [2]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (textid);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [3]:
workdir = r'/home/tari/Загрузки/taiga/Lenta'
filename = 'Lenta.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'newmetadata.csv')
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,segment,textid,textname,textregion,textrubric,textdiff,author,authortexts,authorreaders,magazine,date,time,tags,source
0,Lenta,20150806moonside,НАСА показало обратную сторону Луны на фоне Земли,,Космос,,,,,,6 августа 2015,07:09,,https://lenta.ru/news/2015/08/06/moonside/
1,Lenta,20111205piter2,В центре Петербурга задержаны участники протес...,,Россия,,,,,,5 декабря 2011,23:28,,https://lenta.ru/news/2011/12/05/piter2/
2,Lenta,20161109oilrise,Цена на нефть полностью отыграла потери и нача...,,Госэкономика,,,,,,9 ноября 2016,20:45,,https://lenta.ru/news/2016/11/09/oilrise/
3,Lenta,20121008zhirinovskiy,ЛДПР снимет своего кандидата в пользу брянског...,,Россия,,,,,,8 октября 2012,16:33,,https://lenta.ru/news/2012/10/08/zhirinovskiy/
4,Lenta,20120608platforma,Место проведения светских мероприятий &quot;Ки...,,Россия,,,,,,8 июня 2012,05:09,,https://lenta.ru/news/2012/06/08/platforma/


In [4]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [5]:
meta.iloc[6].to_dict()

{'author': '',
 'authorreaders': '',
 'authortexts': '',
 'date': '5 сентября 2015',
 'magazine': '',
 'segment': 'Lenta',
 'source': 'https://lenta.ru/news/2015/09/05/zvezda/',
 'tags': '',
 'textdiff': '',
 'textid': '20150905zvezda',
 'textname': 'Кремль допустил выделение 20 миллиардов рублей из ФНБ верфи «Звезда»',
 'textregion': '',
 'textrubric': 'Компании',
 'time': '07:31'}

In [ ]:
for i in range(len(meta)):
    values = meta.iloc[i].to_dict()    
    
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    cur.execute(sql, list(values.values()))
    
    valuest = {'id': values['textid'], 'text': unify.open_text(os.path.join(plain, str(values['textid'])+".txt"))}
    
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, list(valuest.values()))
    try:
        valuest2 = {'id': values['textid'], 'text': unify.open_text(open(os.path.join(tagged, str(values['textid'])+".txt"),'r', encoding='utf8').read())}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    except:
        valuest2 = {'id': values['textid'], 'text': ""}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    con.commit()